In [1]:
import pandas as pd
import re

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

import torch

/home/dmitrii/Documents/Work/04_VkusWill/vw_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cleaning data

In [2]:
# Загрузите CSV-файл в DataFrame
df = pd.read_csv('products.csv')

# Функция для замены двойных и более пробелов на одиночные и удаления кавычек
def clean_text(text):
    if isinstance(text, str):
        # Заменить двойные и более пробелы на одиночные
        text = re.sub(r'\s{2,}', ' ', text)
        # Удалить кавычки
        text = text.replace('"', '')
    return text

# Примените функцию ко всем текстовым столбцам
for column in df.select_dtypes(include=['object']):
    df[column] = df[column].apply(clean_text)

# Сохраните DataFrame обратно в CSV
df.to_csv('your_file_cleaned.csv', index=False)

## Detecting volume

In [4]:
# Определяем плотности для категорий продуктов
density = {
    "Фрукты и Овощи": (0.60, 0.80, 0.96),
    "Мясо и Рыба": (1.05, 1.07, 1.10),
    "Молочные Продукты": (1.03, 1.10, 1.10),
    "Хлебобулочные Изделия": (0.27, 0.27, 0.30),
    "Замороженные Продукты": (0.30, 0.90, 0.90),
    "Напитки": (1.00, 1.05, 1.05),
    "Десерты и Сладости": (0.10, 0.70, 0.80),
    "Прочие Продукты": (1.05, 1.05, 1.20)
}

def calculate_volume(weight, category):
    if category not in density:
        raise ValueError("Категория не найдена. Пожалуйста, выберите из следующих: " + ", ".join(density.keys()))

    min_density, avg_density, max_density = density[category]

    # Рассчитываем объем для минимальной, средней и максимальной плотности
    volume_min = weight / min_density
    volume_avg = weight / avg_density
    volume_max = weight / max_density

    return volume_min, volume_avg, volume_max

# Пример использования функции
weight = float(input("Введите вес (кг): "))
category = input("Введите категорию продукта: ")

try:
    volume_min, volume_avg, volume_max = calculate_volume(weight, category)
    print(f"Примерный объем для категории '{category}':")
    print(f"Минимальный объем: {volume_min:.2f} л")
    print(f"Средний объем: {volume_avg:.2f} л")
    print(f"Максимальный объем: {volume_max:.2f} л")
except ValueError as e:
    print(e)

Примерный объем для категории 'Напитки':
Минимальный объем: 1.00 л
Средний объем: 0.95 л
Максимальный объем: 0.95 л


## Detecting category

### Data preprecessing

In [5]:
meat = pd.read_csv('data/scrapy/meat.csv')

In [6]:
meat['category'] = 'meat'

In [7]:
fish = pd.read_csv('data/scrapy/fish.csv')

In [8]:
fish['category'] = 'fish'

In [9]:
df = pd.concat([meat, fish], axis=0, ignore_index=True)

In [10]:
df = df.sample(frac=1).reset_index(drop=True)

In [11]:
df.head()

,title,category
0,"Морская капуста хрустящая со вкусом терияки, 5 г",fish
1,Фарш говяжий спб вес.,meat
2,"Вобла сушеная неразделанная, 100 г",fish
3,Печень говяжья ГП,meat
4,Шейка свиная б/к спб,meat


In [12]:
# Преобразование текстовых меток в числовые
label_encoder = LabelEncoder()
df['category'] = label_encoder.fit_transform(df['category'])

In [13]:
df.head()

,title,category
0,"Морская капуста хрустящая со вкусом терияки, 5 г",0
1,Фарш говяжий спб вес.,1
2,"Вобла сушеная неразделанная, 100 г",0
3,Печень говяжья ГП,1
4,Шейка свиная б/к спб,1


In [14]:
# Разделение на тренировочные и тестовые данные
train_texts, test_texts, train_labels, test_labels = train_test_split(df['title'], df['category'], test_size=0.2, random_state=42)

### Model BERT

Подготовка токенизатора и модели BERT

In [15]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=len(label_encoder.classes_))

/home/dmitrii/Documents/Work/04_VkusWill/vw_env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Data tokenizer

In [16]:
def tokenize_data(texts, labels):
    encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
    return encodings, torch.tensor(labels.tolist())

train_encodings, train_labels = tokenize_data(train_texts, train_labels)
test_encodings, test_labels = tokenize_data(test_texts, test_labels)

#### Create Dataset classes

In [17]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

#### Fit model

In [17]:
training_args = TrainingArguments(
    output_dir='./results',            # Директория для сохранения результатов
    evaluation_strategy="epoch",       # Оценка модели после каждой эпохи
    per_device_train_batch_size=8,     # Размер батча для тренировки
    per_device_eval_batch_size=8,      # Размер батча для оценки
    num_train_epochs=3,                # Количество эпох
    weight_decay=0.01,                 # Коэффициент для L2 регуляризации
    logging_dir='./logs',             # Директория для логов
    logging_steps=10,                  # Шаги между логированием
)

/home/dmitrii/Documents/Work/04_VkusWill/vw_env/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

  3%|▎         | 10/315 [00:17<08:31,  1.68s/it]

{'loss': 0.5393, 'grad_norm': 15.520334243774414, 'learning_rate': 4.841269841269841e-05, 'epoch': 0.1}


  6%|▋         | 20/315 [00:39<10:37,  2.16s/it]

{'loss': 0.4616, 'grad_norm': 12.067536354064941, 'learning_rate': 4.682539682539683e-05, 'epoch': 0.19}


 10%|▉         | 30/315 [01:00<09:59,  2.10s/it]

...

{'loss': 0.0006, 'grad_norm': 0.01460824441164732, 'learning_rate': 2.3809523809523808e-06, 'epoch': 2.86}


 98%|█████████▊| 310/315 [10:55<00:10,  2.05s/it]

{'loss': 0.1405, 'grad_norm': 0.019010959193110466, 'learning_rate': 7.936507936507937e-07, 'epoch': 2.95}


100%|██████████| 315/315 [11:05<00:00,  2.02s/it]

ValueError: You are trying to save a non contiguous tensor: `bert.encoder.layer.0.attention.self.query.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.

In [36]:
# Пример для тензора веса
tensor = tensor.contiguous()

NameError: name 'tensor' is not defined

In [38]:
for name, param in model.named_parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

#### Model estimation

In [19]:
# Оценка модели
predictions = trainer.predict(test_dataset)
predicted_labels = torch.argmax(torch.tensor(predictions.predictions), dim=1)

# Преобразование числовых меток обратно в текстовые категории
predicted_labels_text = label_encoder.inverse_transform(predicted_labels)
test_labels_text = label_encoder.inverse_transform(test_labels)

accuracy = accuracy_score(test_labels_text, predicted_labels_text)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9760765550239234


#### Save model

Обработка тензоров
Если вы работаете с тензорами, которые могли стать не непрерывными из-за модификаций, попробуйте сделать их непрерывными перед сохранением. Пройдитесь по всем параметрам модели и убедитесь, что они являются непрерывными:

In [ ]:
for name, param in model.named_parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()

In [39]:
model.save_pretrained('models/bert/model')
tokenizer.save_pretrained('models/bert/model')

('models/bert/model/tokenizer_config.json',
 'models/bert/model/special_tokens_map.json',
 'models/bert/model/vocab.txt',
 'models/bert/model/added_tokens.json')

#### Load model

In [18]:
tokenizer = BertTokenizer.from_pretrained('models/bert/model')
model = BertForSequenceClassification.from_pretrained('models/bert/model')

#### Get Predict

In [25]:
def preprocess_text(text, tokenizer):
    encoding = tokenizer(text,
                         truncation=True,
                         padding=True,
                         return_tensors='pt',
                         max_length=128)
    return encoding

def predict_category(text, model, tokenizer):
    encoding = preprocess_text(text, tokenizer)
    with torch.no_grad():
        outputs = model(**encoding)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()
    return predicted_class_id

def get_category_name(predicted_class_id, label_encoder):
    return label_encoder.inverse_transform([predicted_class_id])[0]
# Text example
text = "Говядина вес"

# Get predict
predicted_class_id = predict_category(text, model, tokenizer)
category_name = get_category_name(predicted_class_id, label_encoder)

print(f"Predicted category: {category_name}")

Predicted category: meat
